In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import src.graphinglib as gl
from collections import namedtuple
import pyregion
import scipy as sp

from src.tools.messaging import smart_tqdm
from src.hdu.tesseract import Tesseract
from src.hdu.cubes.cube import Cube
from src.hdu.maps.map import Map
from src.hdu.maps.grouped_maps import GroupedMaps

In [ ]:
Info = namedtuple("Info", ["name", "cube_filename", "gaussian_index", "relative_error_threshold"])

files = [
    Info("nii_1", "nii_1_binned_3x3.fits", 1, 0.0016),
    Info("nii_2", "nii_2_binned_4x4.fits", 1, 0.003),
    Info("oiii_1", "oiii_1_binned_3x3.fits", 0, 0.09),
    Info("oiii_2", "oiii_2_binned_4x4.fits", 0, 0.03),
    Info("sii_1", "sii_1_binned_3x3.fits", [0, 1], 0.009),
    Info("sii_2", "sii_2_binned_4x4.fits", [0, 1], 0.016),
    Info("ha_1", "ha_1_binned_3x3.fits", "variable", 0.015),
    Info("ha_2", "ha_2_binned_4x4.fits", 0, 0.005),
]

def reject_outliers(data):
    # Consider 3 sigma values as outliers
    return abs(data - np.nanmedian(data)) < 3 * np.nanstd(data)

def make_fit_histogram(data: np.ndarray, bin_width: float) -> list[gl.Histogram, gl.Curve]:
    data = data.flatten()
    data = np.sort(data[~np.isnan(data)])

    # Create the array of bins, considering that the upper bound is cropped lower to match the bin width
    bins = np.arange(np.nanmin(data), np.nanmax(data) + bin_width*0.99, bin_width)
    hist = np.histogram(data, bins=bins)[0]

    current_plots = [
        gl.Histogram(
            data=data,
            face_color="dimgray",
            edge_color="none",
            normalize=False,
            alpha=1,
            number_of_bins=bins,
            show_params=False
        ),
    ]
    scat = gl.Scatter(
        (bins + bin_width/2)[:-1],
        hist,
    )

    max_x = scat.x_data[np.argmax(hist)]
    fit = gl.FitFromFunction(
        function=lambda x, A, mu, sigma: A* np.exp(-(x - mu)**2 / (2*sigma**2)),
        curve_to_be_fit=scat,
        guesses=[
            np.max(hist)*max_x,
            max_x,
            np.nanstd(data)
        ],
        color="red",
    )
    # fit.label = rf"$m={fit.parameters[1]:.2f}$" + "\n" + rf"$s={abs(fit.parameters[2]):.2f}$"
    fit.label = rf"$s={abs(fit.parameters[2]):.2f}$"
    current_plots.append(fit)

    return current_plots

# Unfiltered statistics

#### Centroid maps creation

In [ ]:
for info in files:
    cube = Cube.load(f"data/orion/data_cubes/binned/{info.cube_filename}")
    tess = Tesseract.load(f"data/orion/fits/{info.name}.fits")
    reg = pyregion.open(f"data/orion/fp_confidence_regions/{info.name}.reg")
    if isinstance(info.gaussian_index, int):
        centroids = tess.to_grouped_maps(["amplitude", "mean", "fwhm_L", "fwhm_G"]).mean[info.gaussian_index]
        centroids = centroids.get_masked_region(reg)
        centroids = centroids.mask((centroids.uncertainties / centroids.data) < info.relative_error_threshold)
        centroids = centroids.mask(reject_outliers(centroids.data))
        centroids = (centroids - 1) * cube.header["CHAN_KMS"] * 1000 + cube.header["VR_CH1"] * 1000
        centroids /= 1000  # convert to km/s
        centroid_maps = [centroids]

    else:  # sii/ha_1
        centroid_maps = []
        if info.name.startswith("sii"):
            mean_maps = tess.to_grouped_maps(["amplitude", "mean", "fwhm_L", "fwhm_G"]).mean
        else:  # ha_1 case
            filtered_tess = tess.filter(slice(18, 25))  # take only the desired components
            mean_maps = filtered_tess.to_grouped_maps(["amplitude", "mean", "fwhm_L", "fwhm_G"]).mean[1:3]
        for centroid in mean_maps:
            centroid = centroid.get_masked_region(reg)
            centroid = centroid.mask((centroid.uncertainties / centroid.data) < info.relative_error_threshold)
            centroid = centroid.mask(reject_outliers(centroid.data))
            centroid_speed = (centroid - 1) * cube.header["CHAN_KMS"] * 1000 + cube.header["VR_CH1"] * 1000
            centroid_speed /= 1000  # convert to km/s
            centroid_maps.append(centroid_speed)

    centroids = GroupedMaps([("centroids", centroid_maps)])
    centroids.save(f"data/orion/centroids/maps/{info.name}_centroids.fits")

#### Separate unfiltered histograms

In [ ]:
for info in files:
    centroids = GroupedMaps.load(f"data/orion/centroids/maps/{info.name}_centroids.fits").centroids
    centroid_data = np.stack([centroid.data for centroid in centroids], axis=0)
    bin_width = np.nanmedian(centroids[0].uncertainties)

    plots = make_fit_histogram(centroid_data, bin_width)
    fig = gl.SmartFigure(
        elements=plots,
        title=f"Centroid {info.name}",
        x_label="Centroid velocity [km s$^{-1}$]",
    ).show()


#### Merged unfiltered histograms

In [ ]:
maps = {}

for info in files:
    centroids = GroupedMaps.load(f"data/orion/centroids/maps/{info.name}_centroids.fits").centroids
    centroid_data = np.stack([centroid.data for centroid in centroids], axis=0)
    centroid_uncertainties = np.stack([centroid.uncertainties for centroid in centroids], axis=0)

    key = info.name.split("_")[0]
    if key not in maps:
        maps[key] = {"data" : [], "uncertainties": []}
    maps[key]["data"].append(centroid_data)
    maps[key]["uncertainties"].append(centroid_uncertainties)

for key, value in maps.items():
    merged_data = np.concatenate([v.flatten() for v in value["data"]])
    merged_uncertainties = np.concatenate([v.flatten() for v in value["uncertainties"]])
    bin_width = np.nanmedian(merged_uncertainties)
    plots = make_fit_histogram(merged_data, bin_width)
    fig = gl.SmartFigure(
        elements=plots[:1],
        title=f"Centroid {key}",
        x_label="Centroid velocity [km s$^{-1}$]",
    ).show()

#### Philippe fun...

In [ ]:
from scipy.constants import c

lambda_channel_1 = 6561.0
lambda_ref = 6562.7797852

print(f"{(lambda_channel_1 - lambda_ref) / lambda_ref * c / 1000} km/s")


In [ ]:
# OIII MADNESS
spec_1 = Cube.load("data/orion/data_cubes/binned/oiii_1_binned_3x3.fits")[:, 100, 100]
spec_2 = Cube.load("data/orion/data_cubes/binned/oiii_2_binned_4x4.fits")[:, 60, 60]

spec_1_plot = spec_1.plot
spec_1_plot.label = "[OIII] Field 1"

spec_2_plot = spec_2.plot
spec_2_plot.label = "[OIII] Field 2"
spec_2_plot.color = gl.get_color(color_number=2)

gl.SmartFigure(
    1,
    2,
    elements=[
        spec_1_plot,
        spec_2_plot,
    ],
    size=(8, 5)
).set_grid().show().save("figures/orion/philippe/oiii_spectrums.pdf")

In [ ]:
# OIII BLOUB
spec_1 = Cube.load("data/orion/data_cubes/binned/oiii_1_binned_3x3.fits")[:, 100, 100]
spec_2 = Cube.load("data/orion/data_cubes/binned/oiii_2_binned_4x4.fits")[:, 60, 60]

spec_1_plot = spec_1.plot
spec_1_plot.label = "[OIII] Field 1"
spec_1_velocities = np.arange(
    spec_1.header["CRVAL1"] / 1000,
    spec_1.header["CRVAL1"] / 1000 + spec_1.header["NAXIS1"] * spec_1.header["CDELT1"] / 1000,
    spec_1.header["CDELT1"] / 1000,
)
spec_1_plot_velocity = spec_1_plot.copy()
spec_1_plot_velocity.x_data = spec_1_velocities
spec_1_wavelengths = np.linspace(
    spec_1.header["WAVE_CH1"],
    spec_1.header["WAVE_LCH"],
    spec_1.header["NAXIS1"],
)
spec_1_plot_wavelength = spec_1_plot.copy()
spec_1_plot_wavelength.x_data = spec_1_wavelengths
spec_1_plot_wavelength.label = None

spec_2_plot = spec_2.plot
spec_2_plot.label = "[OIII] Field 2"
spec_2_plot.color = gl.get_color(color_number=2)
spec_2_velocities = np.arange(
    spec_2.header["CRVAL1"] / 1000,
    spec_2.header["CRVAL1"] / 1000 + spec_2.header["NAXIS1"] * spec_2.header["CDELT1"] / 1000,
    spec_2.header["CDELT1"] / 1000,
)
spec_2_plot_velocity = spec_2_plot.copy()
spec_2_plot_velocity.x_data = spec_2_velocities
spec_2_wavelengths = np.linspace(
    spec_2.header["WAVE_CH1"],
    spec_2.header["WAVE_LCH"],
    spec_2.header["NAXIS1"],
)
spec_2_plot_wavelength = spec_2_plot.copy()
spec_2_plot_wavelength.x_data = spec_2_wavelengths
spec_2_plot_wavelength.label = None

gl.SmartFigure(
    2,
    1,
    elements=[
        [spec_1_plot_velocity, spec_2_plot_velocity],
        [spec_1_plot_wavelength, spec_2_plot_wavelength],
    ],
    size=(6, 5),
    sub_x_labels=["Velocity [km s$^{-1}$]", "Wavelength [Å]"],
    general_legend=True,
    legend_loc="outside lower center",
    legend_cols=2,
).set_grid().show()#.save("figures/orion/philippe/oiii_spectrums_together.pdf")

In [ ]:
# OIII POOP
cubes = [Cube.load(f"data/orion/data_cubes/binned/oiii_{i}_binned_{b}.fits") for i, b in [(1, "3x3"), (2, "4x4")]]
regions = [pyregion.open(f"data/orion/fp_confidence_regions/oiii_{i}.reg") for i in [1, 2]]
spec_wavelengths = [np.linspace(h["WAVE_CH1"], h["WAVE_LCH"], h["NAXIS1"]) for h in [spec_1.header, spec_2.header]]
colors = [gl.get_color(color_number=i) for i in [0, 2]]
spectrums = []

for i, (cube, reg, spec_wavelength, color) in enumerate(zip(cubes, regions, spec_wavelengths, colors)):
    masked_cube = cube.get_masked_region(reg)
    current_spectrums = []
    for line in masked_cube.data.T:
        for pixel in line:
            if np.all(np.isnan(pixel)):
                continue
            current_spectrums.append(gl.Curve(
                spec_wavelength,
                pixel,
                color=color,
                line_width=0.1,
            ))
    spectrums.append(current_spectrums)

gl.SmartFigure(
    elements=[
        spectrums[0],
        spectrums[1],
    ],
    size=(6, 5),
    x_label="Wavelength [Å]",
    legend_loc="outside lower center",
    legend_cols=2,
).set_grid().set_custom_legend([
    gl.LegendLine(
        label="[OIII] Field 1",
        color=gl.get_color(color_number=0),
    ),
    gl.LegendLine(
        label="[OIII] Field 2",
        color=gl.get_color(color_number=2),
    ),
]).show()#.save("figures/orion/philippe/oiii_spectrums_superimposed.pdf")

# Filtered statistics

## Zurflueh filter

See `applications/orion/centroids/statistics.py` for the code used for finding the optimal width

In [ ]:
from src.tools.zurflueh_filter.zfilter import zfilter

In [ ]:
with open("data/orion/centroids/zurflueh_widths.txt", "r") as f:
    lines = f.readlines()

results = {}
current_line = None
for line in lines:
    if "component" in line:
        current_line = line.rstrip("\n")
        continue
    if "===" in line or line == "\n" or "INVALID" in line:
        continue
    results.setdefault(current_line, []).append([int(line[6:9]), float(line.split("=")[-1])])

In [ ]:
optimal_widths = []
for emission_line, data in results.items():
    data = np.array(data)
    optimal_widths.append([emission_line, int(data[np.argmin(data[:, 1]), 0])])

In [ ]:
fig = gl.SmartFigure(
    x_label="Zurflueh Filter Width [pixels]",
    y_label=r"$\Delta F_2(\tau_0)$ [-]",
    elements=[[
        gl.Curve(
            *np.array(data).T,
            label=emission_line,
            line_style="-" if i < 8 else "--",
            line_width=1,
        ),
        gl.Vlines(
            x=optimal_widths[i][1],
            y_min=0,
            y_max=25,
            line_styles="-" if i < 8 else "--",
            line_widths=1,
            colors=gl.get_color(color_number=i%8),
        )]
        for i, (emission_line, data) in enumerate(results.items())
    ],
    size=(16, 9),
    x_lim=(3, 99),
    y_lim=(0, 5)
).set_ticks(
    x_tick_spacing=5,
    minor_x_tick_spacing=1,
    minor_y_tick_spacing=1,
).set_grid(
    which_x="major",
    which_y="major",
)#.show()#.save("figures/orion/centroids/zurflueh_widths_with_lines.pdf")

In [ ]:
figs = []
for field, width in optimal_widths:
    component_i = int(field[-1])
    centroids = GroupedMaps.load(f"data/orion/centroids/maps/{field.split(",")[0]}_centroids.fits").centroids
    centroid_data = centroids[component_i].data
    centroid_uncertainties = centroids[component_i].uncertainties
    filtered_centroids = centroid_data - zfilter(centroid_data, width)
    # gl.SmartFigure(title=field, elements=[gl.Heatmap(filtered_centroids, origin_position="lower")]).show()

    bin_width = np.nanmedian(centroid_uncertainties)
    plots = make_fit_histogram(filtered_centroids, bin_width)
    fig = gl.SmartFigure(
        elements=plots,
        title=field,
        x_label="Centroid velocity [km s$^{-1}$]",
    )#.show()
    figs.append(fig)

fig = gl.SmartFigure(
    3,
    4,
    elements=figs,
    size=(16, 9),
)#.show()#.save("t.pdf")
# fig.show()

In [ ]:
figs = []
optimal_widths_dict = dict(optimal_widths)

load = lambda field: GroupedMaps.load(f"data/orion/centroids/maps/{field}_centroids.fits").centroids

centroid_maps = {
    "[NII]": [
        [load(f"nii_{i}")[0], optimal_widths_dict[f"nii_{i}, component 0"]] for i in [1, 2]
    ],
    "[OIII]": [
        [load(f"oiii_{i}")[0], optimal_widths_dict[f"oiii_{i}, component 0"]] for i in [1, 2]
    ],
    "[SII]": [
        [load("sii_1")[0], optimal_widths_dict["sii_1, component 0"]],
        [load("sii_1")[1], optimal_widths_dict["sii_1, component 1"]],
        [load("sii_2")[0], optimal_widths_dict["sii_2, component 0"]],
        [load("sii_2")[1], optimal_widths_dict["sii_2, component 1"]],
    ],
    r"H$\alpha$": [
        [load("ha_1")[0], optimal_widths_dict["ha_1, component 0"]],
        [load("ha_1")[1], optimal_widths_dict["ha_1, component 1"]],
        [load("ha_2")[0], optimal_widths_dict["ha_2, component 0"]],
    ],
}

all_filtered_centroids = {}
for field, centroids in centroid_maps.items():
    filtered_centroids = [centroid.data - zfilter(centroid.data, width) for centroid, width in centroids]

    bin_width = np.nanmedian(np.concatenate([centroid[0].uncertainties.flatten() for centroid in centroids]))

    plots = make_fit_histogram(np.concatenate([centroid.flatten() for centroid in filtered_centroids]), bin_width)
    figs.append(
        gl.SmartFigure(
            elements=plots,
            subtitles=[field],
            legend_loc="upper right",
            reference_labels=True,
        )
    )
    all_filtered_centroids[field] = filtered_centroids

centroid_fig = gl.SmartFigure(
    2,
    2,
    x_label="Filtered centroid speed [km s$^{-1}$]",
    y_label="Count [pixels]",
    elements=figs,
    size=(8, 8),
)
centroid_fig[0, 0].x_lim = -0.7, 0.7
centroid_fig[0, 1].x_lim = -2, 2
centroid_fig[1, 0].x_lim = -1, 1
centroid_fig[1, 1].x_lim = -1, 1
centroid_fig.set_visual_params(legend_handle_length=1.5)

# centroid_fig.show().save("figures/orion/centroids/filtered_centroids_histograms.pdf", dpi=600)


## Increments

See `applications/orion/centroids/statistics.py` for the code used for this calculation.


## Structure functions